In [1]:
import sys
sys.path.append("..")

In [2]:
import bytelevel

In [3]:
from sklearn.datasets import fetch_20newsgroups
from pprint import pprint
import numpy as np
import pickle

In [4]:
from keras.layers import Embedding, Conv1D, Input, GRU, LSTM, Bidirectional, Dense, UpSampling1D, Dropout, TimeDistributed, RepeatVector
from keras.models import Model

Using TensorFlow backend.


In [5]:
from hyper_params import *
from data_set import *

In [15]:
class RnnCnnHyper(object):
    def __init__(self, embedder, conv, rnn):
        self.embedder = embedder
        self.conv = conv
        self.rnn = rnn
        
    @staticmethod
    def Random(r):
        embedder = EmbeddingHyper.Random(r)
        conv = ConvHyper.Random(r)
        rnn = RnnHyper.Random(r)
        
        return RnnCnnHyper(embedder, conv, rnn)

    def display(self):
        self.embedder.display()
        print()
        self.conv.display()
        print()
        self.rnn.display()
        print()
        
    def make_layers(self, name):
        if name is not None and len(name):
            prefix = '%s_' % name
        else:
            prefix = ''
        embedder = self.embedder.make_layer(name='%sembedder' % prefix)
        conv = self.conv.make_layer(name='%sconv' % prefix)
        rnn = self.rnn.make_layer(name='%srnn' % prefix)
        dense = Dense(self.embedder.vocab_size, activation='softmax', name='%sprobs' % prefix)
        return embedder, conv, rnn, dense


In [7]:
with open('../data/norvig/big.txt') as f:
    data = f.read()

In [8]:
chunks = make_chunks(data)

In [9]:
r = np.random.RandomState(42)

In [10]:
r.shuffle(chunks)
train_text, test_text = make_train_test(chunks)

In [11]:
train, test = SlicedData.Random(train_text, test_text, 64, 1000, r)

In [11]:
def explore(seed, count, max_len):
    r = np.random.RandomState(seed)
    histories = []
    
    def dataset(x):
        x = [random_chop(s, r, max_len + 1) for s in x]
        x = bytelevel.encode(x)
        x = pad_sequences(x, max_len + 1)
        return x

    x_train = dataset(train['data'])
    x_test = dataset(test['data'])
    x_test = x_test[:2000]

    for i in range(count):
        seed = r.randint(200000)
        name = "RnnCnn%d" % seed
        r_i = np.random.RandomState(seed)
        hyper = RnnCnnHyper.Random(r_i)
        hyper.display()

        embed, conv, rnn, pred = hyper.make_layers('', False)

        x = Input(shape=(max_len,), name='text_input')
        h = embed(x)
        print(h.shape)
        h = conv(h)
        print(h.shape)
        h = rnn(h)
        print(h.shape)
        h = pred(h)
        print(h.shape)

        model = Model(x, h)
        model.compile(optimizer='adam',
            loss='sparse_categorical_crossentropy',
            metrics=['sparse_categorical_accuracy'])

        history = model.fit(x=x_train[:, :max_len], 
                     y=x_train[:, max_len],
                    epochs=3, batch_size=10,
                    validation_data=(x_test[:, :max_len], x_test[:, max_len]))

        histories.append([seed, history])
        
        with open("../models/%s.pkl" % name, 'wb') as f:
            pickle.dump([seed, hyper, history.history], f)
            
        model.save("../models/%s.h5" % name)
        
        print()
        print()
        print()
    
    return histories

In [14]:
val_acc = lambda x: x[1].history['val_sparse_categorical_accuracy'][-1]

In [16]:
histories.sort(key=val_acc, reverse=True)

In [17]:
[val_acc(x) for x in histories]

[0.3435000073164701,
 0.3425000065565109,
 0.33200000803917645,
 0.3285000066459179,
 0.3250000074505806,
 0.3230000077188015,
 0.3230000064894557,
 0.323000006377697,
 0.3175000072270632,
 0.3165000065788627,
 0.31350000761449337,
 0.3080000070855021,
 0.3080000065267086,
 0.307500006891787,
 0.305000006519258,
 0.30250000681728123,
 0.2980000066012144,
 0.2505000047758222,
 0.23150000479072333,
 0.22750000536441803,
 0.2275000049546361,
 0.22450000502169132,
 0.2230000051110983,
 0.21600000452250243,
 0.204000004529953,
 0.20300000436604024,
 0.19500000398606063,
 0.19000000402331352,
 0.18750000394880773,
 0.18250000409781933]

In [21]:
def ReHyper(seed):
    r = np.random.RandomState(seed)
    hyper = RnnCnnHyper.Random(r)
    return hyper, r

In [19]:
hyper, _ =ReHyper(histories[0][0])

In [20]:
for seed, _ in histories[:6]:
    hyper, _ = ReHyper(seed)
    print(seed)
    hyper.display()

154351
embedding
vocab size=256
embedding dimension=256

conv 1d
filters=512
kernel size=6
stride = 2

RNN
hidden dimension=512
bidirectional
lstm

150338
embedding
vocab size=256
embedding dimension=128

conv 1d
filters=128
kernel size=5
stride = 1

RNN
hidden dimension=128
bidirectional
gru

17970
embedding
vocab size=256
embedding dimension=256

conv 1d
filters=128
kernel size=2
stride = 2

RNN
hidden dimension=64
bidirectional
gru

154443
embedding
vocab size=256
embedding dimension=128

conv 1d
filters=512
kernel size=6
stride = 4

RNN
hidden dimension=256
bidirectional
gru

27315
embedding
vocab size=256
embedding dimension=128

conv 1d
filters=512
kernel size=8
stride = 3

RNN
hidden dimension=256
bidirectional
gru

62245
embedding
vocab size=256
embedding dimension=512

conv 1d
filters=128
kernel size=2
stride = 4

RNN
hidden dimension=128
gru



In [32]:
max_len = 64

embed_h = EmbeddingHyper(256, 128)
conv_h = ConvHyper(128, 6, 4)
rnn_h = RnnHyper(128, is_lstm=False, is_bidirectional=False, return_sequences=False)
hyper = RnnCnnHyper(embed_h, conv_h, rnn_h)
dec_h = DeconvHyper(128, 6, 4)

dec, up = dec_h.make_layers('')
emb, con, rnn, den = hyper.make_layers('')
dernn = GRU(max_len, return_sequences=True, unroll=True)
tden = TimeDistributed(den)

In [33]:


x = Input(shape=(max_len,), name='text_input')
h = emb(x)
print(h.shape)
h = con(h)
print(h.shape)
h = rnn(h)
print(h.shape)
h = RepeatVector(int(max_len / 4))(h)
print(h.shape)
h = dernn(h)
print(h.shape)
h = up(h)
print(h.shape)
h = dec(h)
print(h.shape)
h = tden(h)
print(h.shape)


(?, 64, 128)
(?, 16, 128)
(?, 128)
(?, 16, 128)
(?, 16, 64)
(?, 64, 64)
(?, 64, 128)
(?, 64, 256)


In [34]:
model = Model(x, h)
model.compile(optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['categorical_accuracy'])

In [36]:
for i in range(100):
    print(i)
    train, test = SlicedData.Random(train_text, test_text, max_len, 10000, r)
    history = model.fit(x=train.x, 
            y=train.y,
            epochs=5, batch_size=100,
            validation_data=(test.x, test.y))
    print()

0
Train on 10000 samples, validate on 1000 samples
Epoch 1/5
10000/10000 [==============================] - 8s - loss: 3.4647 - categorical_accuracy: 0.1499 - val_loss: 3.1045 - val_categorical_accuracy: 0.1624
Epoch 2/5
10000/10000 [==============================] - 7s - loss: 3.1260 - categorical_accuracy: 0.1599 - val_loss: 3.0855 - val_categorical_accuracy: 0.1624
Epoch 3/5
10000/10000 [==============================] - 7s - loss: 3.1077 - categorical_accuracy: 0.1599 - val_loss: 3.0761 - val_categorical_accuracy: 0.1624
Epoch 4/5
10000/10000 [==============================] - 7s - loss: 3.0958 - categorical_accuracy: 0.1599 - val_loss: 3.0691 - val_categorical_accuracy: 0.1624
Epoch 5/5
10000/10000 [==============================] - 7s - loss: 3.0807 - categorical_accuracy: 0.1598 - val_loss: 3.0536 - val_categorical_accuracy: 0.1624

1
Train on 10000 samples, validate on 1000 samples
Epoch 1/5
10000/10000 [==============================] - 7s - loss: 3.0621 - categorical_accuracy

10000/10000 [==============================] - 8s - loss: 2.5309 - categorical_accuracy: 0.2109 - val_loss: 2.5352 - val_categorical_accuracy: 0.2104
Epoch 4/5
10000/10000 [==============================] - 7s - loss: 2.5168 - categorical_accuracy: 0.2133 - val_loss: 2.5286 - val_categorical_accuracy: 0.2112
Epoch 5/5
10000/10000 [==============================] - 7s - loss: 2.5101 - categorical_accuracy: 0.2139 - val_loss: 2.5189 - val_categorical_accuracy: 0.2125

10
Train on 10000 samples, validate on 1000 samples
Epoch 1/5
10000/10000 [==============================] - 7s - loss: 2.5161 - categorical_accuracy: 0.2134 - val_loss: 2.5195 - val_categorical_accuracy: 0.2151
Epoch 2/5
10000/10000 [==============================] - 7s - loss: 2.4960 - categorical_accuracy: 0.2167 - val_loss: 2.4941 - val_categorical_accuracy: 0.2183
Epoch 3/5
10000/10000 [==============================] - 7s - loss: 2.4830 - categorical_accuracy: 0.2189 - val_loss: 2.4857 - val_categorical_accuracy: 0.22

10000/10000 [==============================] - 7s - loss: 2.1103 - categorical_accuracy: 0.3235 - val_loss: 2.1143 - val_categorical_accuracy: 0.3201
Epoch 2/5
10000/10000 [==============================] - 7s - loss: 2.0904 - categorical_accuracy: 0.3289 - val_loss: 2.1156 - val_categorical_accuracy: 0.3230
Epoch 3/5
10000/10000 [==============================] - 7s - loss: 2.0768 - categorical_accuracy: 0.3339 - val_loss: 2.1000 - val_categorical_accuracy: 0.3280
Epoch 4/5
10000/10000 [==============================] - 7s - loss: 2.0653 - categorical_accuracy: 0.3380 - val_loss: 2.0930 - val_categorical_accuracy: 0.3317
Epoch 5/5
10000/10000 [==============================] - 7s - loss: 2.0514 - categorical_accuracy: 0.3436 - val_loss: 2.0874 - val_categorical_accuracy: 0.3359

20
Train on 10000 samples, validate on 1000 samples
Epoch 1/5
10000/10000 [==============================] - 7s - loss: 2.0747 - categorical_accuracy: 0.3403 - val_loss: 2.0816 - val_categorical_accuracy: 0.33

10000/10000 [==============================] - 7s - loss: 1.5718 - categorical_accuracy: 0.5041 - val_loss: 1.5950 - val_categorical_accuracy: 0.4955
Epoch 5/5
10000/10000 [==============================] - 7s - loss: 1.5596 - categorical_accuracy: 0.5077 - val_loss: 1.5777 - val_categorical_accuracy: 0.5009

29
Train on 10000 samples, validate on 1000 samples
Epoch 1/5
10000/10000 [==============================] - 7s - loss: 1.5785 - categorical_accuracy: 0.5025 - val_loss: 1.5779 - val_categorical_accuracy: 0.5029
Epoch 2/5
10000/10000 [==============================] - 7s - loss: 1.5591 - categorical_accuracy: 0.5080 - val_loss: 1.5594 - val_categorical_accuracy: 0.5089
Epoch 3/5
10000/10000 [==============================] - 7s - loss: 1.5452 - categorical_accuracy: 0.5116 - val_loss: 1.5460 - val_categorical_accuracy: 0.5117
Epoch 4/5
10000/10000 [==============================] - 7s - loss: 1.5301 - categorical_accuracy: 0.5156 - val_loss: 1.5519 - val_categorical_accuracy: 0.50

10000/10000 [==============================] - 8s - loss: 1.2984 - categorical_accuracy: 0.5836 - val_loss: 1.2996 - val_categorical_accuracy: 0.5821
Epoch 2/5
10000/10000 [==============================] - 7s - loss: 1.2844 - categorical_accuracy: 0.5876 - val_loss: 1.2971 - val_categorical_accuracy: 0.5838
Epoch 3/5
10000/10000 [==============================] - 7s - loss: 1.2740 - categorical_accuracy: 0.5905 - val_loss: 1.2951 - val_categorical_accuracy: 0.5835
Epoch 4/5
10000/10000 [==============================] - 7s - loss: 1.2680 - categorical_accuracy: 0.5927 - val_loss: 1.2916 - val_categorical_accuracy: 0.5849
Epoch 5/5
10000/10000 [==============================] - 7s - loss: 1.2617 - categorical_accuracy: 0.5938 - val_loss: 1.2940 - val_categorical_accuracy: 0.5842

39
Train on 10000 samples, validate on 1000 samples
Epoch 1/5
10000/10000 [==============================] - 7s - loss: 1.2852 - categorical_accuracy: 0.5875 - val_loss: 1.2732 - val_categorical_accuracy: 0.58

...

10000/10000 [==============================] - 7s - loss: 1.0488 - categorical_accuracy: 0.6605 - val_loss: 1.0586 - val_categorical_accuracy: 0.6559
Epoch 2/5
10000/10000 [==============================] - 7s - loss: 1.0364 - categorical_accuracy: 0.6639 - val_loss: 1.0551 - val_categorical_accuracy: 0.6568
Epoch 3/5
10000/10000 [==============================] - 7s - loss: 1.0277 - categorical_accuracy: 0.6661 - val_loss: 1.0521 - val_categorical_accuracy: 0.6584
Epoch 4/5
10000/10000 [==============================] - 7s - loss: 1.0196 - categorical_accuracy: 0.6686 - val_loss: 1.0456 - val_categorical_accuracy: 0.6587
Epoch 5/5
10000/10000 [==============================] - 7s - loss: 1.0154 - categorical_accuracy: 0.6706 - val_loss: 1.0459 - val_categorical_accuracy: 0.6588

58
Train on 10000 samples, validate on 1000 samples
Epoch 1/5
10000/10000 [==============================] - 7s - loss: 1.0452 - categorical_accuracy: 0.6616 - val_loss: 1.0493 - val_categorical_accuracy: 0.65

10000/10000 [==============================] - 7s - loss: 0.9269 - categorical_accuracy: 0.6985 - val_loss: 0.9651 - val_categorical_accuracy: 0.6854
Epoch 5/5
10000/10000 [==============================] - 7s - loss: 0.9193 - categorical_accuracy: 0.7011 - val_loss: 0.9653 - val_categorical_accuracy: 0.6857

67
Train on 10000 samples, validate on 1000 samples
Epoch 1/5
10000/10000 [==============================] - 7s - loss: 0.9542 - categorical_accuracy: 0.6907 - val_loss: 0.9471 - val_categorical_accuracy: 0.6934
Epoch 2/5
10000/10000 [==============================] - 7s - loss: 0.9371 - categorical_accuracy: 0.6958 - val_loss: 0.9606 - val_categorical_accuracy: 0.6874
Epoch 3/5
10000/10000 [==============================] - 7s - loss: 0.9218 - categorical_accuracy: 0.7001 - val_loss: 0.9483 - val_categorical_accuracy: 0.6902
Epoch 4/5
10000/10000 [==============================] - 7s - loss: 0.9154 - categorical_accuracy: 0.7023 - val_loss: 0.9376 - val_categorical_accuracy: 0.69

10000/10000 [==============================] - 7s - loss: 0.8245 - categorical_accuracy: 0.7338 - val_loss: 0.8240 - val_categorical_accuracy: 0.7339
Epoch 2/5
10000/10000 [==============================] - 7s - loss: 0.8032 - categorical_accuracy: 0.7406 - val_loss: 0.8097 - val_categorical_accuracy: 0.7374
Epoch 3/5
10000/10000 [==============================] - 7s - loss: 0.7941 - categorical_accuracy: 0.7435 - val_loss: 0.8014 - val_categorical_accuracy: 0.7424
Epoch 4/5
10000/10000 [==============================] - 7s - loss: 0.7888 - categorical_accuracy: 0.7448 - val_loss: 0.8099 - val_categorical_accuracy: 0.7387
Epoch 5/5
10000/10000 [==============================] - 7s - loss: 0.7781 - categorical_accuracy: 0.7487 - val_loss: 0.7937 - val_categorical_accuracy: 0.7448

77
Train on 10000 samples, validate on 1000 samples
Epoch 1/5
10000/10000 [==============================] - 7s - loss: 0.8031 - categorical_accuracy: 0.7405 - val_loss: 0.8013 - val_categorical_accuracy: 0.74

10000/10000 [==============================] - 7s - loss: 0.6290 - categorical_accuracy: 0.8001 - val_loss: 0.6525 - val_categorical_accuracy: 0.7909
Epoch 5/5
10000/10000 [==============================] - 7s - loss: 0.6170 - categorical_accuracy: 0.8044 - val_loss: 0.6382 - val_categorical_accuracy: 0.7963

86
Train on 10000 samples, validate on 1000 samples
Epoch 1/5
10000/10000 [==============================] - 7s - loss: 0.6504 - categorical_accuracy: 0.7930 - val_loss: 0.6528 - val_categorical_accuracy: 0.7924
Epoch 2/5
10000/10000 [==============================] - 7s - loss: 0.6369 - categorical_accuracy: 0.7979 - val_loss: 0.6588 - val_categorical_accuracy: 0.7883
Epoch 3/5
10000/10000 [==============================] - 7s - loss: 0.6308 - categorical_accuracy: 0.7996 - val_loss: 0.6648 - val_categorical_accuracy: 0.7865
Epoch 4/5
10000/10000 [==============================] - 7s - loss: 0.6179 - categorical_accuracy: 0.8044 - val_loss: 0.6606 - val_categorical_accuracy: 0.78

10000/10000 [==============================] - 8s - loss: 0.5752 - categorical_accuracy: 0.8188 - val_loss: 0.5917 - val_categorical_accuracy: 0.8109
Epoch 2/5
10000/10000 [==============================] - 7s - loss: 0.5582 - categorical_accuracy: 0.8255 - val_loss: 0.5807 - val_categorical_accuracy: 0.8157
Epoch 3/5
10000/10000 [==============================] - 7s - loss: 0.5466 - categorical_accuracy: 0.8296 - val_loss: 0.5559 - val_categorical_accuracy: 0.8266
Epoch 4/5
10000/10000 [==============================] - 7s - loss: 0.5412 - categorical_accuracy: 0.8316 - val_loss: 0.5657 - val_categorical_accuracy: 0.8223
Epoch 5/5
10000/10000 [==============================] - 7s - loss: 0.5391 - categorical_accuracy: 0.8324 - val_loss: 0.5690 - val_categorical_accuracy: 0.8201

96
Train on 10000 samples, validate on 1000 samples
Epoch 1/5
10000/10000 [==============================] - 7s - loss: 0.5626 - categorical_accuracy: 0.8243 - val_loss: 0.5521 - val_categorical_accuracy: 0.82

In [37]:
for i in range(100):
    print(i + 100)
    train, test = SlicedData.Random(train_text, test_text, max_len, 10000, r)
    history = model.fit(x=train.x, 
            y=train.y,
            epochs=5, batch_size=100,
            validation_data=(test.x, test.y))
    print()

100
Train on 10000 samples, validate on 1000 samples
Epoch 1/5
10000/10000 [==============================] - 7s - loss: 0.5452 - categorical_accuracy: 0.8294 - val_loss: 0.5526 - val_categorical_accuracy: 0.8252
Epoch 2/5
10000/10000 [==============================] - 7s - loss: 0.5262 - categorical_accuracy: 0.8364 - val_loss: 0.5328 - val_categorical_accuracy: 0.8351
Epoch 3/5
10000/10000 [==============================] - 7s - loss: 0.5152 - categorical_accuracy: 0.8411 - val_loss: 0.5300 - val_categorical_accuracy: 0.8361
Epoch 4/5
10000/10000 [==============================] - 7s - loss: 0.5173 - categorical_accuracy: 0.8396 - val_loss: 0.5401 - val_categorical_accuracy: 0.8316
Epoch 5/5
10000/10000 [==============================] - 7s - loss: 0.5073 - categorical_accuracy: 0.8432 - val_loss: 0.5308 - val_categorical_accuracy: 0.8346

101
Train on 10000 samples, validate on 1000 samples
Epoch 1/5
10000/10000 [==============================] - 8s - loss: 0.5379 - categorical_accu

10000/10000 [==============================] - 8s - loss: 0.4777 - categorical_accuracy: 0.8528 - val_loss: 0.4846 - val_categorical_accuracy: 0.8525
Epoch 4/5
10000/10000 [==============================] - 7s - loss: 0.4695 - categorical_accuracy: 0.8565 - val_loss: 0.4853 - val_categorical_accuracy: 0.8523
Epoch 5/5
10000/10000 [==============================] - 7s - loss: 0.4700 - categorical_accuracy: 0.8552 - val_loss: 0.4878 - val_categorical_accuracy: 0.8495

110
Train on 10000 samples, validate on 1000 samples
Epoch 1/5
10000/10000 [==============================] - 7s - loss: 0.4971 - categorical_accuracy: 0.8463 - val_loss: 0.5056 - val_categorical_accuracy: 0.8410
Epoch 2/5
10000/10000 [==============================] - 7s - loss: 0.4827 - categorical_accuracy: 0.8510 - val_loss: 0.4986 - val_categorical_accuracy: 0.8446
Epoch 3/5
10000/10000 [==============================] - 7s - loss: 0.4629 - categorical_accuracy: 0.8588 - val_loss: 0.4796 - val_categorical_accuracy: 0.8

10000/10000 [==============================] - 7s - loss: 0.4625 - categorical_accuracy: 0.8570 - val_loss: 0.4509 - val_categorical_accuracy: 0.8597
Epoch 2/5
10000/10000 [==============================] - 7s - loss: 0.4382 - categorical_accuracy: 0.8669 - val_loss: 0.4476 - val_categorical_accuracy: 0.8595
Epoch 3/5
10000/10000 [==============================] - 7s - loss: 0.4353 - categorical_accuracy: 0.8675 - val_loss: 0.4542 - val_categorical_accuracy: 0.8591
Epoch 4/5
10000/10000 [==============================] - 7s - loss: 0.4359 - categorical_accuracy: 0.8664 - val_loss: 0.4512 - val_categorical_accuracy: 0.8594
Epoch 5/5
10000/10000 [==============================] - 7s - loss: 0.4290 - categorical_accuracy: 0.8692 - val_loss: 0.4547 - val_categorical_accuracy: 0.8582

120
Train on 10000 samples, validate on 1000 samples
Epoch 1/5
10000/10000 [==============================] - 7s - loss: 0.4553 - categorical_accuracy: 0.8604 - val_loss: 0.4580 - val_categorical_accuracy: 0.8

10000/10000 [==============================] - 7s - loss: 0.4027 - categorical_accuracy: 0.8779 - val_loss: 0.4182 - val_categorical_accuracy: 0.8711
Epoch 5/5
10000/10000 [==============================] - 7s - loss: 0.4026 - categorical_accuracy: 0.8774 - val_loss: 0.4295 - val_categorical_accuracy: 0.8669

129
Train on 10000 samples, validate on 1000 samples
Epoch 1/5
10000/10000 [==============================] - 7s - loss: 0.4395 - categorical_accuracy: 0.8642 - val_loss: 0.4348 - val_categorical_accuracy: 0.8668
Epoch 2/5
10000/10000 [==============================] - 7s - loss: 0.4142 - categorical_accuracy: 0.8741 - val_loss: 0.4139 - val_categorical_accuracy: 0.8754
Epoch 3/5
10000/10000 [==============================] - 7s - loss: 0.4110 - categorical_accuracy: 0.8748 - val_loss: 0.4360 - val_categorical_accuracy: 0.8665
Epoch 4/5
10000/10000 [==============================] - 7s - loss: 0.4012 - categorical_accuracy: 0.8787 - val_loss: 0.4414 - val_categorical_accuracy: 0.8

10000/10000 [==============================] - 8s - loss: 0.4118 - categorical_accuracy: 0.8734 - val_loss: 0.4085 - val_categorical_accuracy: 0.8756
Epoch 2/5
10000/10000 [==============================] - 7s - loss: 0.3905 - categorical_accuracy: 0.8816 - val_loss: 0.4032 - val_categorical_accuracy: 0.8772
Epoch 3/5
10000/10000 [==============================] - 7s - loss: 0.3845 - categorical_accuracy: 0.8840 - val_loss: 0.4161 - val_categorical_accuracy: 0.8702
Epoch 4/5
10000/10000 [==============================] - 7s - loss: 0.3907 - categorical_accuracy: 0.8804 - val_loss: 0.4090 - val_categorical_accuracy: 0.8740
Epoch 5/5
10000/10000 [==============================] - 7s - loss: 0.3821 - categorical_accuracy: 0.8840 - val_loss: 0.4230 - val_categorical_accuracy: 0.8688

139
Train on 10000 samples, validate on 1000 samples
Epoch 1/5
10000/10000 [==============================] - 8s - loss: 0.4004 - categorical_accuracy: 0.8784 - val_loss: 0.3878 - val_categorical_accuracy: 0.8

...

10000/10000 [==============================] - 8s - loss: 0.3730 - categorical_accuracy: 0.8868 - val_loss: 0.3738 - val_categorical_accuracy: 0.8856
Epoch 2/5
10000/10000 [==============================] - 7s - loss: 0.3550 - categorical_accuracy: 0.8936 - val_loss: 0.3711 - val_categorical_accuracy: 0.8876
Epoch 3/5
10000/10000 [==============================] - 7s - loss: 0.3456 - categorical_accuracy: 0.8968 - val_loss: 0.3553 - val_categorical_accuracy: 0.8929
Epoch 4/5
10000/10000 [==============================] - 7s - loss: 0.3471 - categorical_accuracy: 0.8959 - val_loss: 0.3980 - val_categorical_accuracy: 0.8761
Epoch 5/5
10000/10000 [==============================] - 7s - loss: 0.3555 - categorical_accuracy: 0.8916 - val_loss: 0.3910 - val_categorical_accuracy: 0.8769

158
Train on 10000 samples, validate on 1000 samples
Epoch 1/5
10000/10000 [==============================] - 8s - loss: 0.4142 - categorical_accuracy: 0.8704 - val_loss: 0.4161 - val_categorical_accuracy: 0.8

10000/10000 [==============================] - 7s - loss: 0.3339 - categorical_accuracy: 0.9005 - val_loss: 0.3441 - val_categorical_accuracy: 0.8968
Epoch 5/5
10000/10000 [==============================] - 7s - loss: 0.3277 - categorical_accuracy: 0.9026 - val_loss: 0.3557 - val_categorical_accuracy: 0.8926

167
Train on 10000 samples, validate on 1000 samples
Epoch 1/5
10000/10000 [==============================] - 7s - loss: 0.3674 - categorical_accuracy: 0.8886 - val_loss: 0.3937 - val_categorical_accuracy: 0.8774
Epoch 2/5
10000/10000 [==============================] - 7s - loss: 0.3691 - categorical_accuracy: 0.8869 - val_loss: 0.3479 - val_categorical_accuracy: 0.8960
Epoch 3/5
10000/10000 [==============================] - 8s - loss: 0.3267 - categorical_accuracy: 0.9039 - val_loss: 0.3410 - val_categorical_accuracy: 0.8995
Epoch 4/5
10000/10000 [==============================] - 8s - loss: 0.3322 - categorical_accuracy: 0.9016 - val_loss: 0.3423 - val_categorical_accuracy: 0.8

10000/10000 [==============================] - 8s - loss: 0.3666 - categorical_accuracy: 0.8877 - val_loss: 0.3744 - val_categorical_accuracy: 0.8851
Epoch 2/5
10000/10000 [==============================] - 7s - loss: 0.3363 - categorical_accuracy: 0.8991 - val_loss: 0.3463 - val_categorical_accuracy: 0.8971
Epoch 3/5
10000/10000 [==============================] - 7s - loss: 0.3136 - categorical_accuracy: 0.9084 - val_loss: 0.3385 - val_categorical_accuracy: 0.8990
Epoch 4/5
10000/10000 [==============================] - 7s - loss: 0.3107 - categorical_accuracy: 0.9090 - val_loss: 0.3300 - val_categorical_accuracy: 0.9035
Epoch 5/5
10000/10000 [==============================] - 7s - loss: 0.3113 - categorical_accuracy: 0.9085 - val_loss: 0.3380 - val_categorical_accuracy: 0.8999

177
Train on 10000 samples, validate on 1000 samples
Epoch 1/5
10000/10000 [==============================] - 8s - loss: 0.3347 - categorical_accuracy: 0.9003 - val_loss: 0.3461 - val_categorical_accuracy: 0.8

10000/10000 [==============================] - 7s - loss: 0.3235 - categorical_accuracy: 0.9028 - val_loss: 0.3542 - val_categorical_accuracy: 0.8915
Epoch 5/5
10000/10000 [==============================] - 7s - loss: 0.3475 - categorical_accuracy: 0.8936 - val_loss: 0.3393 - val_categorical_accuracy: 0.8987

186
Train on 10000 samples, validate on 1000 samples
Epoch 1/5
10000/10000 [==============================] - 7s - loss: 0.4185 - categorical_accuracy: 0.8711 - val_loss: 0.3857 - val_categorical_accuracy: 0.8829
Epoch 2/5
10000/10000 [==============================] - 7s - loss: 0.3287 - categorical_accuracy: 0.9023 - val_loss: 0.3120 - val_categorical_accuracy: 0.9084
Epoch 3/5
10000/10000 [==============================] - 8s - loss: 0.2951 - categorical_accuracy: 0.9156 - val_loss: 0.3148 - val_categorical_accuracy: 0.9082
Epoch 4/5
10000/10000 [==============================] - 8s - loss: 0.2913 - categorical_accuracy: 0.9166 - val_loss: 0.3118 - val_categorical_accuracy: 0.9

10000/10000 [==============================] - 8s - loss: 0.3173 - categorical_accuracy: 0.9055 - val_loss: 0.4234 - val_categorical_accuracy: 0.8863
Epoch 2/5
10000/10000 [==============================] - 7s - loss: 0.3108 - categorical_accuracy: 0.9077 - val_loss: 0.3082 - val_categorical_accuracy: 0.9093
Epoch 3/5
10000/10000 [==============================] - 7s - loss: 0.2964 - categorical_accuracy: 0.9129 - val_loss: 0.3005 - val_categorical_accuracy: 0.9110
Epoch 4/5
10000/10000 [==============================] - 7s - loss: 0.2867 - categorical_accuracy: 0.9164 - val_loss: 0.3100 - val_categorical_accuracy: 0.9079
Epoch 5/5
10000/10000 [==============================] - 7s - loss: 0.2869 - categorical_accuracy: 0.9165 - val_loss: 0.3255 - val_categorical_accuracy: 0.9023

196
Train on 10000 samples, validate on 1000 samples
Epoch 1/5
10000/10000 [==============================] - 8s - loss: 0.3812 - categorical_accuracy: 0.8824 - val_loss: 0.3838 - val_categorical_accuracy: 0.8

In [38]:
test.text[:4]

['lf." And indeed he only had to lie down, to fall asleep like a\ns',
 "ount was much annoyed. 'How could\nyou have written it yourself?'",
 "you fool.' And what do you think? 'No,'\nsaid he, 'I have not rea",
 'nesses.\n\n"The hours are the same, and the lathe, and also the ma']

In [39]:
bytelevel.prediction2str(model.predict(test.x[:4]))

['ly." And indeed he only had to lie dowe, to fall apleep like ao ',
 'ount was much annoyed. Thow hould\nyou maye written it yoursel   ',
 'you fool.\' An" what do you thin\'? \'eot\' said \'e, \'e ha\'e not ae ',
 'nesses.\n\n"The hours are the same, and the lathe, and anso the   ']

In [ ]:
for i in range(200):
    print(i + 200)
    train, test = SlicedData.Random(train_text, test_text, max_len, 10000, r)
    history = model.fit(x=train.x, 
            y=train.y,
            epochs=5, batch_size=100,
            validation_data=(test.x, test.y))
    print()

200
Train on 10000 samples, validate on 1000 samples
Epoch 1/5
10000/10000 [==============================] - 7s - loss: 0.3006 - categorical_accuracy: 0.9124 - val_loss: 0.3072 - val_categorical_accuracy: 0.9095
Epoch 2/5
10000/10000 [==============================] - 7s - loss: 0.2866 - categorical_accuracy: 0.9171 - val_loss: 0.3018 - val_categorical_accuracy: 0.9103
Epoch 3/5
10000/10000 [==============================] - 7s - loss: 0.2971 - categorical_accuracy: 0.9124 - val_loss: 0.3348 - val_categorical_accuracy: 0.8966
Epoch 4/5
10000/10000 [==============================] - 7s - loss: 0.2890 - categorical_accuracy: 0.9153 - val_loss: 0.3010 - val_categorical_accuracy: 0.9100
Epoch 5/5
10000/10000 [==============================] - 7s - loss: 0.2840 - categorical_accuracy: 0.9175 - val_loss: 0.3178 - val_categorical_accuracy: 0.9027

201
Train on 10000 samples, validate on 1000 samples
Epoch 1/5
10000/10000 [==============================] - 7s - loss: 0.3455 - categorical_accu

10000/10000 [==============================] - 7s - loss: 0.2754 - categorical_accuracy: 0.9204 - val_loss: 0.2919 - val_categorical_accuracy: 0.9129
Epoch 4/5
10000/10000 [==============================] - 7s - loss: 0.2846 - categorical_accuracy: 0.9165 - val_loss: 0.3276 - val_categorical_accuracy: 0.9001
Epoch 5/5
10000/10000 [==============================] - 7s - loss: 0.3647 - categorical_accuracy: 0.8877 - val_loss: 0.4200 - val_categorical_accuracy: 0.8697

210
Train on 10000 samples, validate on 1000 samples
Epoch 1/5
10000/10000 [==============================] - 7s - loss: 0.3365 - categorical_accuracy: 0.8980 - val_loss: 0.2938 - val_categorical_accuracy: 0.9138
Epoch 2/5
10000/10000 [==============================] - 7s - loss: 0.2744 - categorical_accuracy: 0.9215 - val_loss: 0.2815 - val_categorical_accuracy: 0.9188
Epoch 3/5
10000/10000 [==============================] - 7s - loss: 0.2721 - categorical_accuracy: 0.9223 - val_loss: 0.3036 - val_categorical_accuracy: 0.9

10000/10000 [==============================] - 7s - loss: 0.2820 - categorical_accuracy: 0.9181 - val_loss: 0.2886 - val_categorical_accuracy: 0.9157
Epoch 2/5
10000/10000 [==============================] - 7s - loss: 0.2713 - categorical_accuracy: 0.9218 - val_loss: 0.2867 - val_categorical_accuracy: 0.9155
Epoch 3/5
10000/10000 [==============================] - 7s - loss: 0.2746 - categorical_accuracy: 0.9199 - val_loss: 0.2850 - val_categorical_accuracy: 0.9165
Epoch 4/5
10000/10000 [==============================] - 7s - loss: 0.2663 - categorical_accuracy: 0.9232 - val_loss: 0.2956 - val_categorical_accuracy: 0.9119
Epoch 5/5
10000/10000 [==============================] - 7s - loss: 0.2777 - categorical_accuracy: 0.9188 - val_loss: 0.3058 - val_categorical_accuracy: 0.9074

220
Train on 10000 samples, validate on 1000 samples
Epoch 1/5
10000/10000 [==============================] - 7s - loss: 0.3490 - categorical_accuracy: 0.8935 - val_loss: 0.3284 - val_categorical_accuracy: 0.8